## 라이브러리 import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn           as sb
import scipy.stats       as sp

In [2]:
import tensorflow as tf
from tensorflow import keras
tf.random.set_seed(777)

스케일러 라이브러리

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import joblib

## 데이터 불러오기

In [5]:
inputdata = pd.read_csv('./input-test1.csv',skiprows = 6, sep=',')
inputdata

,Name,P1,P2,P3,P4,P5,P6,P7,P8,P10
0,DP 0,80,120.0,110.0,100.0,98.0,85.0,80.0,80.0,0.002414
1,DP 1,80,80.2,105.4,103.8,114.2,82.2,99.8,100.6,0.002639
2,DP 2,80,80.6,83.8,97.0,105.8,86.2,113.4,81.8,0.002597
3,DP 3,80,81.0,93.8,115.0,118.6,105.4,98.2,98.6,0.002549
4,DP 4,80,81.4,107.8,87.4,98.6,108.2,87.8,91.8,0.002536
...,...,...,...,...,...,...,...,...,...,...
96,DP 96,80,118.2,108.2,116.6,117.0,106.2,94.6,82.6,0.002395
97,DP 97,80,118.6,109.8,80.6,109.8,107.0,99.4,94.2,0.002536
98,DP 98,80,119.0,104.6,109.8,111.0,89.4,93.0,97.0,0.002611
99,DP 99,80,119.4,80.6,100.2,119.4,109.8,109.4,89.8,0.002535


# training, label 분리

In [11]:
TrainData = inputdata.iloc[:,1:-1]

TrainLabel_ = inputdata.iloc[1:,9]


In [12]:
pd.set_option('display.max_rows',10)
TrainData

,P1,P2,P3,P4,P5,P6,P7,P8
0,80,120.0,110.0,100.0,98.0,85.0,80.0,80.0
1,80,80.2,105.4,103.8,114.2,82.2,99.8,100.6
2,80,80.6,83.8,97.0,105.8,86.2,113.4,81.8
3,80,81.0,93.8,115.0,118.6,105.4,98.2,98.6
4,80,81.4,107.8,87.4,98.6,108.2,87.8,91.8
...,...,...,...,...,...,...,...,...
96,80,118.2,108.2,116.6,117.0,106.2,94.6,82.6
97,80,118.6,109.8,80.6,109.8,107.0,99.4,94.2
98,80,119.0,104.6,109.8,111.0,89.4,93.0,97.0
99,80,119.4,80.6,100.2,119.4,109.8,109.4,89.8


In [13]:
pd.DataFrame(TrainLabel_)

,P10
1,0.002639
2,0.002597
3,0.002549
4,0.002536
5,0.002529
...,...
96,0.002395
97,0.002536
98,0.002611
99,0.002535


In [14]:
max_value = np.max(TrainLabel_)
min_value = np.min(TrainLabel_)

dist_value = max_value - min_value

TrainLabel = ((TrainLabel_ - min_value)/dist_value)+0.5

In [15]:
pd.set_option("Display.max_rows",10)
pd.DataFrame(TrainLabel)

,P10
1,1.184604
2,1.068196
3,0.931607
4,0.897109
5,0.875775
...,...
96,0.500000
97,0.896182
98,1.106091
99,0.894975


## ANN hyperparameter 조절에 따른 학습성능 확인 

#### 지정 iteration마다 학습과정 확인 함수(Class) 정의

In [10]:
EpochForPrint = 100

class AccuracyPerEpoch(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        keras.callbacks.Callback()
        if epoch%EpochForPrint == 0:
            print("[{} Epochs]    RMSE:{:.5f},   MAE: {:.5f},  MAPE: {:.2f}%"
                  .format(epoch, np.sqrt(logs['mse']), logs['mae'], logs['mape']))

#### Hyperparameter 조합 설정

In [11]:
# 조정 하이퍼파라미터 : 학습율, 은닉층 뉴런 수
Lr = [0.001, 0.005, 0.01]   # Learning Rates
N1 = [30, 40, 50, 60, 70]   # Number of Neurons on Hidden Layer 1
N2 = [10, 20, 30]           # Number of Neurons on Hidden Layer 2

Model = ['thickness']

# 고정 하이퍼파라미터 : 입력/출력층 뉴런 수, 학습 Epoch 수
noOfNeuron_in  = 50
noOfNeuron_out = 1
Epoch          = 2000

print('Number of case : %d'%(len(Lr)*len(N1)*len(N2)))

Number of case : 45


### 모델 학습 1 - FFT 특징 기반 Wn1, Wn2, R1, R2 예측

In [12]:
for M in range(1):
    
    Tr_result_temp = np.zeros((len(Lr)*len(N1)*len(N2) , 7)) # *len(N2)
    cnt = 0
    
#     exec('Label_Trn = TrainLabel_%d'%(M+1))
    print('\n\n\n\n################## Model %d (Predict :'%(M+1) + Model[M] + ') ##################')

    for i in range(len(Lr)):
        learningRate = Lr[i]

        for j in range(len(N1)):
            noOfNeuron1 = N1[j]
            
            for k in range(len(N2)):
                noOfNeuron2 = N2[k]

                print('\n\nTrial No.%d'%(cnt+1))
                print('Prediction :' + Model[M])
                print('Learning rate : {:.3}'.format(learningRate))
                print('Hidden 1 neuron : %d'%(noOfNeuron1))
                print('Hidden 2 neuron : %d'%(noOfNeuron2))

                ################ 신경망 구조 재설계 ################

                tf.keras.backend.clear_session()
                def ANN_model(input_data):
                    model = keras.Sequential()
                    model.add(keras.layers.Dense(units = noOfNeuron_in,
                                                 input_shape = (input_data.shape[1],), activation = 'relu'))  # Input  Layer
                    model.add(keras.layers.Dense(units = noOfNeuron1,                  activation = 'relu'))  # Hidden Layer 1
                    model.add(keras.layers.Dense(units = noOfNeuron2,                  activation = 'relu'))  # Hidden Layer 2
                    model.add(keras.layers.Dense(units = noOfNeuron_out,             )) # Output Layer
                    model.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate),
                                  loss=keras.losses.mean_absolute_error,
                                  metrics=['mse','mae','mape'])
                    return model
                model = ANN_model(TrainData)

                ################ 신경망 학습 ################

                hist = model.fit(TrainData, TrainLabel, epochs=Epoch, verbose=0, callbacks=[AccuracyPerEpoch()], batch_size=100)
                print("\n[Final Epochs]    RMSE:{:.5f},   MAE: {:.5f},  MAPE: {:.2f}%"
                      .format(np.sqrt(hist.history['mse'][-1]), hist.history['mae'][-1], hist.history['mape'][-1]))
                
                model.save('D:/testoneblow/MLmodels/Model_%d.h5'%(cnt+1))
                
                Tr_result_temp[cnt,0] = cnt+1
                Tr_result_temp[cnt,1] = learningRate
                Tr_result_temp[cnt,2] = noOfNeuron1
                Tr_result_temp[cnt,3] = noOfNeuron2
                Tr_result_temp[cnt,4] = np.sqrt(hist.history['mse'][-1])
                Tr_result_temp[cnt,5] = hist.history['mae'][-1]
                Tr_result_temp[cnt,6] = hist.history['mape'][-1]

                cnt=cnt+1


    Tr_result_temp_pd = pd.DataFrame(Tr_result_temp, columns=['Case', 'L.rate', 'Nr-HL1', 'Nr-HL2', 'RMSE', 'MAE', 'MAPE'])
    Tr_result_temp_pd.to_csv('D:/testoneblow/ANN_prediction1.0/Tr_result_epoch%d.csv'%(Epoch), index=None)





################## Model 1 (Predict :thickness) ##################


Trial No.1
Prediction :thickness
Learning rate : 0.001
Hidden 1 neuron : 30
Hidden 2 neuron : 10
[0 Epochs]    RMSE:27.00732,   MAE: 26.94053,  MAPE: 2549.66%
[100 Epochs]    RMSE:0.49849,   MAE: 0.39288,  MAPE: 37.67%
[200 Epochs]    RMSE:0.22845,   MAE: 0.19174,  MAPE: 18.68%
[300 Epochs]    RMSE:0.25580,   MAE: 0.22107,  MAPE: 21.85%
[400 Epochs]    RMSE:0.49104,   MAE: 0.43751,  MAPE: 38.46%
[500 Epochs]    RMSE:0.33381,   MAE: 0.30051,  MAPE: 30.45%
[600 Epochs]    RMSE:0.27755,   MAE: 0.24100,  MAPE: 24.48%
[700 Epochs]    RMSE:0.22663,   MAE: 0.17942,  MAPE: 15.84%
[800 Epochs]    RMSE:0.19349,   MAE: 0.16640,  MAPE: 16.23%
[900 Epochs]    RMSE:0.16648,   MAE: 0.13071,  MAPE: 11.90%
[1000 Epochs]    RMSE:0.24022,   MAE: 0.20847,  MAPE: 19.11%
[1100 Epochs]    RMSE:0.44736,   MAE: 0.40902,  MAPE: 36.35%
[1200 Epochs]    RMSE:0.20719,   MAE: 0.16391,  MAPE: 14.08%
[1300 Epochs]    RMSE:0.35386,   MAE: 0.32865



Trial No.7
Prediction :thickness
Learning rate : 0.001
Hidden 1 neuron : 50
Hidden 2 neuron : 10
[0 Epochs]    RMSE:5.32187,   MAE: 4.96267,  MAPE: 440.16%
[100 Epochs]    RMSE:0.48935,   MAE: 0.38472,  MAPE: 39.60%
[200 Epochs]    RMSE:0.48891,   MAE: 0.45426,  MAPE: 41.70%
[300 Epochs]    RMSE:0.58226,   MAE: 0.50739,  MAPE: 47.40%
[400 Epochs]    RMSE:0.43276,   MAE: 0.40019,  MAPE: 36.10%
[500 Epochs]    RMSE:0.51634,   MAE: 0.48694,  MAPE: 44.66%
[600 Epochs]    RMSE:0.85249,   MAE: 0.83595,  MAPE: 80.99%
[700 Epochs]    RMSE:0.27454,   MAE: 0.22552,  MAPE: 20.95%
[800 Epochs]    RMSE:0.34750,   MAE: 0.29783,  MAPE: 27.32%
[900 Epochs]    RMSE:0.19099,   MAE: 0.15210,  MAPE: 14.45%
[1000 Epochs]    RMSE:0.19257,   MAE: 0.15773,  MAPE: 16.96%
[1100 Epochs]    RMSE:0.29582,   MAE: 0.24578,  MAPE: 22.64%
[1200 Epochs]    RMSE:0.15752,   MAE: 0.12131,  MAPE: 12.99%
[1300 Epochs]    RMSE:0.16052,   MAE: 0.13037,  MAPE: 12.70%
[1400 Epochs]    RMSE:0.24037,   MAE: 0.21251,  MAPE: 21.0

[0 Epochs]    RMSE:18.69745,   MAE: 18.45579,  MAPE: 1740.63%
[100 Epochs]    RMSE:0.62593,   MAE: 0.53266,  MAPE: 48.59%
[200 Epochs]    RMSE:0.63360,   MAE: 0.57581,  MAPE: 54.03%
[300 Epochs]    RMSE:0.66792,   MAE: 0.61855,  MAPE: 58.65%
[400 Epochs]    RMSE:1.22039,   MAE: 1.20224,  MAPE: 113.06%
[500 Epochs]    RMSE:0.75970,   MAE: 0.73605,  MAPE: 70.41%
[600 Epochs]    RMSE:0.17163,   MAE: 0.13584,  MAPE: 12.61%
[700 Epochs]    RMSE:0.77379,   MAE: 0.73875,  MAPE: 69.43%
[800 Epochs]    RMSE:0.84061,   MAE: 0.80883,  MAPE: 78.59%
[900 Epochs]    RMSE:0.47075,   MAE: 0.44353,  MAPE: 42.86%
[1000 Epochs]    RMSE:0.19410,   MAE: 0.15542,  MAPE: 14.71%
[1100 Epochs]    RMSE:0.12975,   MAE: 0.09770,  MAPE: 9.79%
[1200 Epochs]    RMSE:0.37074,   MAE: 0.33028,  MAPE: 33.46%
[1300 Epochs]    RMSE:0.28510,   MAE: 0.26331,  MAPE: 26.25%
[1400 Epochs]    RMSE:0.14817,   MAE: 0.11899,  MAPE: 11.33%
[1500 Epochs]    RMSE:0.17768,   MAE: 0.14942,  MAPE: 14.28%
[1600 Epochs]    RMSE:0.50264,  

[0 Epochs]    RMSE:14.20314,   MAE: 14.08568,  MAPE: 1332.68%
[100 Epochs]    RMSE:0.30097,   MAE: 0.25603,  MAPE: 22.53%
[200 Epochs]    RMSE:0.24416,   MAE: 0.20728,  MAPE: 21.43%
[300 Epochs]    RMSE:0.24406,   MAE: 0.20715,  MAPE: 21.36%
[400 Epochs]    RMSE:0.24428,   MAE: 0.20705,  MAPE: 21.07%
[500 Epochs]    RMSE:0.24448,   MAE: 0.20748,  MAPE: 21.60%
[600 Epochs]    RMSE:0.24417,   MAE: 0.20696,  MAPE: 21.12%
[700 Epochs]    RMSE:0.24442,   MAE: 0.20714,  MAPE: 21.03%
[800 Epochs]    RMSE:0.24554,   MAE: 0.20786,  MAPE: 21.87%
[900 Epochs]    RMSE:0.24919,   MAE: 0.21100,  MAPE: 20.68%
[1000 Epochs]    RMSE:0.24438,   MAE: 0.20742,  MAPE: 21.56%
[1100 Epochs]    RMSE:0.24440,   MAE: 0.20744,  MAPE: 21.57%
[1200 Epochs]    RMSE:0.24693,   MAE: 0.20819,  MAPE: 22.10%
[1300 Epochs]    RMSE:0.24816,   MAE: 0.20840,  MAPE: 22.26%
[1400 Epochs]    RMSE:0.24841,   MAE: 0.21023,  MAPE: 20.68%
[1500 Epochs]    RMSE:0.24654,   MAE: 0.20854,  MAPE: 20.74%
[1600 Epochs]    RMSE:0.24471,  

[0 Epochs]    RMSE:11.61067,   MAE: 11.40174,  MAPE: 1084.05%
[100 Epochs]    RMSE:0.30897,   MAE: 0.27042,  MAPE: 28.95%
[200 Epochs]    RMSE:0.13935,   MAE: 0.10886,  MAPE: 11.69%
[300 Epochs]    RMSE:0.16716,   MAE: 0.13957,  MAPE: 14.26%
[400 Epochs]    RMSE:0.14521,   MAE: 0.11980,  MAPE: 11.70%
[500 Epochs]    RMSE:0.34160,   MAE: 0.32701,  MAPE: 32.34%
[600 Epochs]    RMSE:0.23234,   MAE: 0.20110,  MAPE: 21.59%
[700 Epochs]    RMSE:0.17760,   MAE: 0.14119,  MAPE: 12.97%
[800 Epochs]    RMSE:0.16253,   MAE: 0.14340,  MAPE: 14.59%
[900 Epochs]    RMSE:0.11065,   MAE: 0.08726,  MAPE: 8.80%
[1000 Epochs]    RMSE:0.12527,   MAE: 0.09562,  MAPE: 8.71%
[1100 Epochs]    RMSE:0.10858,   MAE: 0.08896,  MAPE: 8.14%
[1200 Epochs]    RMSE:0.15781,   MAE: 0.11180,  MAPE: 13.34%
[1300 Epochs]    RMSE:0.13000,   MAE: 0.09901,  MAPE: 11.29%
[1400 Epochs]    RMSE:0.12670,   MAE: 0.09860,  MAPE: 8.99%
[1500 Epochs]    RMSE:0.10599,   MAE: 0.08411,  MAPE: 7.66%
[1600 Epochs]    RMSE:0.08647,   MAE:

[0 Epochs]    RMSE:18.02082,   MAE: 17.79813,  MAPE: 1706.93%
[100 Epochs]    RMSE:0.31363,   MAE: 0.27468,  MAPE: 28.11%
[200 Epochs]    RMSE:0.16401,   MAE: 0.13288,  MAPE: 14.41%
[300 Epochs]    RMSE:0.16570,   MAE: 0.13338,  MAPE: 14.19%
[400 Epochs]    RMSE:0.42678,   MAE: 0.38306,  MAPE: 39.32%
[500 Epochs]    RMSE:0.27588,   MAE: 0.22893,  MAPE: 25.17%
[600 Epochs]    RMSE:0.15284,   MAE: 0.12395,  MAPE: 12.13%
[700 Epochs]    RMSE:0.14633,   MAE: 0.11927,  MAPE: 12.36%
[800 Epochs]    RMSE:0.13009,   MAE: 0.10226,  MAPE: 9.97%
[900 Epochs]    RMSE:0.12637,   MAE: 0.09971,  MAPE: 9.83%
[1000 Epochs]    RMSE:0.12903,   MAE: 0.10280,  MAPE: 10.15%
[1100 Epochs]    RMSE:0.16306,   MAE: 0.13745,  MAPE: 14.60%
[1200 Epochs]    RMSE:0.14773,   MAE: 0.11621,  MAPE: 13.04%
[1300 Epochs]    RMSE:0.23784,   MAE: 0.21333,  MAPE: 22.21%
[1400 Epochs]    RMSE:0.12354,   MAE: 0.09628,  MAPE: 10.30%
[1500 Epochs]    RMSE:0.12720,   MAE: 0.09856,  MAPE: 10.79%
[1600 Epochs]    RMSE:0.12547,   M

[0 Epochs]    RMSE:24.57046,   MAE: 24.30017,  MAPE: 2296.15%
[100 Epochs]    RMSE:0.33375,   MAE: 0.28833,  MAPE: 31.27%
[200 Epochs]    RMSE:0.21469,   MAE: 0.15671,  MAPE: 18.46%
[300 Epochs]    RMSE:0.23374,   MAE: 0.18260,  MAPE: 20.70%
[400 Epochs]    RMSE:0.18908,   MAE: 0.14010,  MAPE: 16.40%
[500 Epochs]    RMSE:0.19786,   MAE: 0.15714,  MAPE: 17.74%
[600 Epochs]    RMSE:0.25596,   MAE: 0.22973,  MAPE: 24.19%
[700 Epochs]    RMSE:0.17737,   MAE: 0.14877,  MAPE: 12.97%
[800 Epochs]    RMSE:0.12150,   MAE: 0.09570,  MAPE: 9.48%
[900 Epochs]    RMSE:0.11949,   MAE: 0.09378,  MAPE: 9.56%
[1000 Epochs]    RMSE:0.14638,   MAE: 0.11978,  MAPE: 12.74%
[1100 Epochs]    RMSE:0.15058,   MAE: 0.12344,  MAPE: 11.35%
[1200 Epochs]    RMSE:0.14408,   MAE: 0.11970,  MAPE: 12.51%
[1300 Epochs]    RMSE:0.31399,   MAE: 0.29266,  MAPE: 29.07%
[1400 Epochs]    RMSE:0.16985,   MAE: 0.14552,  MAPE: 13.58%
[1500 Epochs]    RMSE:0.11914,   MAE: 0.08926,  MAPE: 9.80%
[1600 Epochs]    RMSE:0.21971,   MA

[0 Epochs]    RMSE:31.33909,   MAE: 31.24628,  MAPE: 2942.10%
[100 Epochs]    RMSE:0.32190,   MAE: 0.26059,  MAPE: 29.18%
[200 Epochs]    RMSE:0.14378,   MAE: 0.11522,  MAPE: 11.28%
[300 Epochs]    RMSE:0.22913,   MAE: 0.18290,  MAPE: 20.53%
[400 Epochs]    RMSE:0.23014,   MAE: 0.19989,  MAPE: 17.20%
[500 Epochs]    RMSE:0.24457,   MAE: 0.21812,  MAPE: 23.02%
[600 Epochs]    RMSE:0.13016,   MAE: 0.10474,  MAPE: 11.42%
[700 Epochs]    RMSE:0.17187,   MAE: 0.13706,  MAPE: 12.12%
[800 Epochs]    RMSE:0.11693,   MAE: 0.09196,  MAPE: 9.72%
[900 Epochs]    RMSE:0.19031,   MAE: 0.16245,  MAPE: 17.48%
[1000 Epochs]    RMSE:0.12875,   MAE: 0.09787,  MAPE: 8.89%
[1100 Epochs]    RMSE:0.17881,   MAE: 0.15593,  MAPE: 13.98%
[1200 Epochs]    RMSE:0.16973,   MAE: 0.14603,  MAPE: 13.33%
[1300 Epochs]    RMSE:0.13471,   MAE: 0.11216,  MAPE: 10.83%
[1400 Epochs]    RMSE:0.18746,   MAE: 0.16567,  MAPE: 17.23%
[1500 Epochs]    RMSE:0.13506,   MAE: 0.11071,  MAPE: 10.53%
[1600 Epochs]    RMSE:0.14944,   M

In [ ]:
Tr_result_temp_pd.to_csv('D:/testoneblow/ANN_prediction1.0/Tr_result1.csv', index=None)

### [Wn1, Wn2, R1, R2] 최고성능 모델 재학습 및 모델 & 히스토리 저장

In [ ]:
for M in range(1):

    Tr_result_temp = pd.read_csv('D:/testoneblow/ANN_prediction1.0/Tr_result_epoch20000.csv')
    learningRate   = Tr_result_temp.sort_values(['MAE'],ascending=True).iloc[0,1]
    noOfNeuron1    = np.int(Tr_result_temp.sort_values(['MAE'],ascending=True).iloc[0,2])
    noOfNeuron2    = np.int(Tr_result_temp.sort_values(['MAE'],ascending=True).iloc[0,3])
    Epoch          = 20000
    
    print('\n\n\nPrediction :' + Model[M])
    print('Learning rate : {:.3}'.format(learningRate))
    print('Hidden 1 neuron : %d'%(noOfNeuron1))
    print('Hidden 2 neuron : %d'%(noOfNeuron2))
    
#     exec('Label_Trn = TrainLabel_%d'%(M+1))
    
    ################ 신경망 구조 재설계 ################

    tf.keras.backend.clear_session()
    def ANN_model(input_data):
        model = keras.Sequential()
        model.add(keras.layers.Dense(units = noOfNeuron_in,
                                     input_shape = (input_data.shape[1],), activation = 'relu'))  # Input  Layer
        model.add(keras.layers.Dense(units = noOfNeuron1,                  activation = 'relu'))  # Hidden Layer 1
        model.add(keras.layers.Dense(units = noOfNeuron2,                  activation = 'relu'))  # Hidden Layer 2
        model.add(keras.layers.Dense(units = noOfNeuron_out,               )) # Output Layer
        model.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate),
                      loss=keras.losses.mean_absolute_error,
                      metrics=['mse','mae','mape'])
        return model
    model = ANN_model(TrainData)

    ################ 신경망 학습 ################

    BestModel_temp = model.fit(TrainData, TrainLabel, epochs=Epoch, verbose=0, callbacks=[AccuracyPerEpoch()], batch_size=100)
    print("\n[Final Epochs]    RMSE:{:.5f},   MAE: {:.5f},  MAPE: {:.2f}%"
          .format(np.sqrt(BestModel_temp.history['mse'][-1]), BestModel_temp.history['mae'][-1], BestModel_temp.history['mape'][-1]))
    
    # 모델 저장
    model.save('D:/testoneblow/MLmodels/BestModel_2DOF_0410_M%d.h5'%(M+1))
    
    # 히스토리 저장
    RMSE  = np.sqrt(np.array(BestModel_temp.history['mse'])[:, np.newaxis])
    MAE   = np.array(BestModel_temp.history['mae'])[:, np.newaxis]
    MAPE  = np.array(BestModel_temp.history['mape'])[:, np.newaxis]

    History_temp = pd.DataFrame(np.concatenate([RMSE,MAE,MAPE],axis=1))
    History_temp.to_csv("D:/testoneblow/MLmodels/BestModel_2DOF_0410_M%d_history.csv"%(M+1), index=None)

In [ ]:
plt.plot(BestModel_temp.history['mape'])

plt.ylabel("mape", fontsize='16')
plt.xlabel("epoch", fontsize='16')
plt.ylim(0,100)

In [ ]:
(model.predict(TrainData)-0.5)*dist_value+min_value

In [ ]:
type(TrainLabel_)

In [ ]:
ErrorTable = np.zeros((101,3))
ErrorTable[:,0] = TrainLabel_
for i in range(101):
    a = ((model.predict(TrainData)-0.5)*dist_value+min_value)[i]
    ErrorTable[i,1] = a
    ErrorTable[i,2] = ((TrainLabel_[i]-a)/TrainLabel_[i])*100

In [ ]:
ErrorTable = pd.DataFrame(ErrorTable)

In [ ]:
ErrorTable.to_csv('C:/Users/SDML/Desktop/ErrorTable.csv')